# Quick Start

In this notebook:
* Setup ApertureDB database server 
* Setup a Google Colab notebook with aperturedb client
* Then configure it
* Load a dataset ApertureDB server
* Run Queries!

## 1. Set up the Database

ApertureDB is set up as a database (server) and can be accessed from clients anywhere as long as the server is accessible on the network to the client.

Sign up for an Aperture [cloud account here](https://cloud.aperturedata.io) (30 days free trial) or see [other methods here](http://docs.aperturedata.io/category/setup-server)

## 2. Set up the Client

### a) Launch Colab or Jupyter Server

* Run any of our notebooks on Google Colab or your own Jupyter server as indicated from Step 2.b) onwards.
* You can use our [pre-configured Jupyter server](https://docs.aperturedata.io/Setup/client/notebooks?notebooks=notebook) 
* This jupyter lab launches on port 8888 which will already have the SDK installed.
  
```bash
docker run --interactive --tty aperturedata/aperturedb-notebook
```

### b) Install ApertureDB SDK

In [ ]:
%pip install --quiet --upgrade aperturedb

### c) Define server configuration

The easiest way to set up a configuration is to paste a JSON configuration string into `adb config create`.
See [Configuration](https://docs.aperturedata.io/Setup/client/configuration) for how to get this string and for alternative ways to set up a configuration.

In [ ]:
! adb config create --active --from-json

Enter JSON string: 

## 3. Connect ApertureDB Client and Server

As a demonstration that you have connected to the server successfully, let's see a summary of the database schema.  
The first time you do this, you may need to grant permission for this notebook to access your secrets.

In [ ]:
from aperturedb.Utils import Utils
from aperturedb.CommonLibrary import create_connector

# Create the connector for ApertureDB
client = create_connector()

# Use the connector to create a Utils object and print the summary
utils = Utils(client)
utils.summary()

## 4. Ingest Our Example Cookbook Data

>**Cookbook Dataset** is a dataset comprising of Dishes(their photos, descriptions), mapped to the constituent main ingredients.
>
>It is a small Dataset built to show the capabilities of ApertureDB.

* [Cookbook Dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?gid=0#gid=0)
* [Git Repository](https://github.com/aperture-data/Cookbook)
* [More details on how to ingest](https://docs.aperturedata.io/HowToGuides/Ingestion/Ingestion/Examples/Loading_CSVParsers)

In [ ]:
! wget https://github.com/aperture-data/Cookbook/raw/refs/heads/main/scripts/load_cookbook_data.sh
! bash load_cookbook_data.sh


from aperturedb.cli.ingest import from_csv, TransformerType, IngestType

from_csv(
    filepath="data/dishes.adb.csv",
    ingest_type=IngestType.IMAGE,
    transformer=[
      TransformerType.clip_pytorch_embeddings,
      TransformerType.image_properties,
      TransformerType.common_properties]
    )

from_csv(
    filepath="data/ingredients.adb.csv",
    ingest_type=IngestType.ENTITY,
)

from_csv(
    filepath="data/dish_ingredients.adb.csv",
    ingest_type=IngestType.CONNECTION,
)

## 5. Run Queries!

### a) Find images of dishes from Scottish Cuisine

For this simple query, we are able to use an object wrapper class.

In [ ]:
from aperturedb.Images import Images
from aperturedb.Constraints import Constraints

# Create an Images wrapper object
images = Images(client)

# Search
images.search(
    constraints=Constraints().equal("cuisine", "Scottish"),
    limit=3,
)

# display result
images.display()

### b) Find images and information of dishes with butter and chicken in them

In [ ]:
from aperturedb.CommonLibrary import create_connector, execute_query
from aperturedb.NotebookHelpers import display

query = [
  { # Find an ingredient with the name "butter"
    "FindEntity": {
      "with_class": "Ingredient",
      "constraints": {
        "name": [
          "==",
          "butter"
        ]
      },
      "_ref": 1, # forward reference
    }
  },
  { # Find an ingredient with the name "chicken"
    "FindEntity": {
      "with_class": "Ingredient",
      "constraints": {
        "name": [
          "==",
          "chicken"
        ]
      },
      "_ref": 2 # forward reference
    }
  },
  { # Find an image connected to both ingredients
    "FindImage": {
      "blobs": True,
      "is_connected_to": {
        "all": [ # Must be connected to all ingredients
          {
            "ref": 1, # backward reference
          },
          {
            "ref": 2, # backward reference
          }
        ]
      },
      "results": {
        "limit": 5, # Maximum of five results
        "all_properties": True # Return all properties
      }
    }
  }
]
client = create_connector()
result, response, blobs = execute_query(client, query, [])

# Print the response
print(response)

# Check if the query was successful
if result == 0:
  display(blobs)


### c) Find similar looking dishes to a bread

In [ ]:
from aperturedb.CommonLibrary import create_connector, execute_query
from aperturedb.NotebookHelpers import display
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

search_tokens = clip.tokenize([f"a photo of bread"]).to(device)
search_embeddings = model.encode_text(search_tokens)

query = [{
    "FindDescriptor": {
        "set": "ViT-B/16", # Name of descriptor set to use
        "k_neighbors": 5, # How many results to return
        "distances": True, # Also return distances (as _distance property)
        "blobs": False, # Don't return vectors of matching descriptors
        "_ref": 1 # forward reference
    }
}, {
    "FindImage": {
        "blobs": True, # Return image binary data
        "is_connected_to": {
            "ref": 1 # backward reference
        },
        "results": {
            "all_properties": True # Return all properties
        }
    }
}]

if device == "cuda":
    search_embeddings = search_embeddings.float()
    blobs = search_embeddings[0].cpu().detach().numpy().tobytes()
else:
    blobs = search_embeddings[0].detach().numpy().tobytes()

client = create_connector()
result, response, blobs = execute_query(client, query, [blobs])

# Print the response
print(response)

# Check if the query was successful
if result == 0:
  display(blobs)


# What's Next?
* Think about how to move from your current data tools to ApertureDB - [learn more](https://docs.aperturedata.io/category/database-concepts) about the unique features and differences
* Learn to add and [work with various objects](https://docs.aperturedata.io/category/start-with-basics) in your instance using our Cookbook dataset
* [Bulk load](https://docs.aperturedata.io/category/manage-multimodal-datasets) various data types